In [7]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import pydae.svg_tools as svgt
#%config InlineBackend.figure_format = 'svg'
import pydae.grid_tools as gt
import scipy.optimize as sopt
from scipy.optimize import NonlinearConstraint


In [8]:
%matplotlib widget

In [9]:
from pydae import ssa
from oc_3bus import oc_3bus_class

In [14]:
#grid_wo = oc_3bus_class()
#grid_pi = oc_3bus_vsg_pi_class()

grid = oc_3bus_class()


params = {"S_n_B1":100e3,"S_n_B3":100e3,"K_p_agc":0.0,"K_i_agc":1,
          "p_c_B1":0.0,"p_c_B3":0.0,
          "R_v_B1":0.0,"R_v_B3":0.0,
          "X_v_B1":0.1,"X_v_B3":0.1,
          'v_ref_B1': 1.049999999986537,
          'v_ref_B3': 1.049610220464389,
          "Droop_B1":0.05,"Droop_B3":0.05,
          "K_delta_B1":0.0,"K_delta_B3":0.00001,
          "P_B2":-50e3,"Q_B2":0e3,
          "T_v_B1":0.1,"T_v_B3":0.1,
          "K_sec_B1":0.5,"K_sec_B3":0.5 }



In [15]:
grid = oc_3bus_class()

gt.change_line(grid,'B1','B2',X_km=0.167,R_km=0.287,km=0.1)
gt.change_line(grid,'B2','B3',X_km=0.167,R_km=0.287,km=0.3)

grid.initialize([params],'xy_0.json',compile=True)
grid.report_y()
#grid.save_0('xy_0.json')

#ssa.eval_A(grid_pi)
#ssa.damp_report(grid_pi)
#ssa.participation(grid)
P_loss = grid.get_value('p_g_B1')*100e3 + grid.get_value('p_g_B3')*100e3 + grid.get_value('P_B2') 

print(P_loss)

V_B1  =  1.05
theta_B1 = -0.03
V_B2  =  1.04
theta_B2 = -0.03
V_B3  =  1.05
theta_B3 = -0.02
omega_B1 =  1.00
i_d_B1 =  0.04
i_q_B1 =  0.24
p_g_B1 =  0.25
q_g_B1 =  0.04
p_m_B1 =  0.25
omega_B3 =  1.00
i_d_B3 = -0.03
i_q_B3 =  0.24
p_g_B3 =  0.25
q_g_B3 = -0.03
p_m_B3 =  0.25
omega_coi =  1.00
p_agc =  0.50
420.78860825540323


In [16]:
grid = oc_3bus_class()
gt.change_line(grid,'B1','B2',X_km=0.167,R_km=0.287,km=0.1)
gt.change_line(grid,'B2','B3',X_km=0.167,R_km=0.287,km=0.3)
grid.initialize([params],'xy_0.json',compile=True)
print(f'P_B2: {grid.get_value("P_B2"):0.2f}')
grid.run([{'t_end': 1.0,'Dt':0.01,'decimation':1}])
grid.run([{'t_end': 5.0,'P_B2':-100e3}])
grid.run([{'t_end': 8.0,'v_ref_B1':1.032,'v_ref_B3':1.05}])
grid.run([{'t_end':30.0,'omega_ref_B1':1.011,'omega_ref_B3':1.011}])
#grid.set_value('P_B2', -1e3)
#grid.run([{'t_end':3.0}])
grid.post();

P_B2: -50000.00


In [17]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(4, 4))
axes[0].plot(grid.T,grid.get_values('p_g_B1'), label='$p_{B1}$')
axes[0].plot(grid.T,grid.get_values('p_g_B3'), label='$p_{B3}$')
axes[1].plot(grid.T,grid.get_values('q_g_B1'), label='$q_{B1}$')
axes[1].plot(grid.T,grid.get_values('q_g_B3'), label='$q_{B3}$')

for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')

axes[1].legend(loc='best')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(4, 5))
axes[0].plot(grid.T,grid.get_values('omega_B1'), label='$\omega_{B1}$')
axes[0].plot(grid.T,grid.get_values('omega_B3'), label='$\omega_{B3}$')
axes[0].plot(grid.T,grid.get_values('omega_coi'), label='$\omega_{coi}$')
axes[1].plot(grid.T,grid.get_values('V_B2'), label='$V_{{B2}}$')

P_loss = grid.get_values('p_g_B1')*100e3 + grid.get_values('p_g_B3')*100e3 + grid.get_values('P_B2') 
axes[2].plot(grid.T,P_loss, label='$P_{{loss}}$ (W)')

for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')

axes[1].legend(loc='best')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Optmizer

In [158]:
class opt_class:
    
    def __init__(self,grid):
        self.grid = grid    
        self.x_0 = np.array([ 1,1 ])
        self.bounds = [(0.95,1.05),(0.95,1.05)]
        
    def optimize(self):
            
        #res = sopt.minimize(opt.obj_eval, x0, method='trust-constr', bounds=bounds)  # SLSQP, Powell, COBYLA,trust-constr
        cons_1 = {'type':'ineq','fun':opt.cons_1}
        cons = [cons_1]
        res = sopt.minimize(opt.obj_eval, self.x_0, method='trust-constr', bounds=self.bounds, 
                            constraints=cons)  # SLSQP, Powell, COBYLA,trust-constr
        self.res = res
        self.x_0 = res.x 
        return res.x
        
    
    def obj_eval(self,u):
        self.grid.load_0('xy_0.json')

        v_ref_B1 = u[0]
        v_ref_B3 = u[1]
        
        self.grid.set_value('v_ref_B1',v_ref_B1)
        self.grid.set_value('v_ref_B3',v_ref_B3) 
        
        self.grid.ss()

        P_B1 = self.grid.get_value('p_g_B1')*100e3
        P_B3 = self.grid.get_value('p_g_B3')*100e3
        P_B2 = self.grid.get_value('P_B2')

        P_loss = P_B1 + P_B3 + P_B2
        return P_loss

    def cons(self,u):
         
        return {'type':'ineq','fun':self.cons_1(u)}

    def cons_1(self,u):
        self.grid.load_0('xy_0.json')

        v_ref_B1 = u[0]
        v_ref_B3 = u[1]
        
        self.grid.set_value('v_ref_B1',v_ref_B1)
        self.grid.set_value('v_ref_B3',v_ref_B3) 
        
        self.grid.ss()

        V_B3 = self.grid.get_value('V_B3')
        
        return 1.05-V_B3
    
opt = opt_class(grid_ss)

u = [1.1,1]
opt.obj_eval(u)
opt.cons(u)
opt.optimize()
opt.res

 barrier_parameter: 0.0008000000000000003
 barrier_tolerance: 0.0008000000000000003
          cg_niter: 24
      cg_stop_cond: 4
            constr: [array([7.8078868e-06]), array([1.0459524 , 1.04999088])]
       constr_nfev: [45, 0]
       constr_nhev: [0, 0]
       constr_njev: [0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.12965607643127441
               fun: 102.73856691579567
              grad: array([ -48.47205585, -149.29770708])
               jac: [array([[-0.41904048, -0.57953092]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([-2.45784298e-09,  4.34795311e-10])
           message: '`gtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 45
              nhev: 0
               nit: 18
             niter: 18
              njev: 15
        optimality: 2.4578429846933147e-09
            status: 1
           success: True
         tr_radius: 1190.2502223809092
                 v: [a

## Control loop

In [168]:
grid = oc_3bus_class()    # for time domain simulation
grid_ss = oc_3bus_class() # for steady state optimization
gt.change_line(grid,'B1','B2',X_km=0.167,R_km=0.287,km=0.1)
gt.change_line(grid,'B2','B3',X_km=0.167,R_km=0.287,km=0.3)
gt.change_line(grid_ss,'B1','B2',X_km=0.167,R_km=0.287,km=0.1)
gt.change_line(grid_ss,'B2','B3',X_km=0.167,R_km=0.287,km=0.3)

params_ctrl = params.copy()
params_ctrl.update({'Dt':0.01,'decimation':1})
grid.initialize([params_ctrl],'xy_0.json',compile=True)
grid_ss.initialize([params_ctrl],'xy_0.json',compile=True)

P_B2_0 = grid.get_value('P_B2')

opt = opt_class(grid_ss)
opt.grid.set_value('P_B2',P_B2_0)
v_ref_B1, v_ref_B3 = opt.optimize()

params_ctrl.update({'v_ref_B1':v_ref_B1,'v_ref_B3':v_ref_B3})
grid.initialize([params_ctrl],'xy_0.json',compile=True)

Δt = 1.0
t_end = 10.0

times = np.arange(0,t_end+Δt,Δt)

for t in times:
    
    # events
    P_B2 = P_B2_0
    if t>2.0:
        P_B2 = 2*P_B2_0
    if t>7.0:
        P_B2 = 0.5*P_B2_0
        
    # simulate one large step
    grid.run([{'t_end': t,'P_B2':P_B2}])
    
    # update ss system
    opt.grid.set_value('P_B2',P_B2)
    
    # update inputs in main system
    v_ref_B1, v_ref_B3 = opt.optimize()      

    grid.set_value('v_ref_B1',v_ref_B1)
    grid.set_value('v_ref_B3',v_ref_B3) 
        
    

grid.post();

In [169]:
fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(4, 8))

axes[0].plot(grid.T,grid.get_values('p_g_B1'), label='$p_{B1}$')
axes[0].plot(grid.T,grid.get_values('p_g_B3'), label='$p_{B3}$')

axes[1].plot(grid.T,grid.get_values('q_g_B1'), label='$q_{B1}$')
axes[1].plot(grid.T,grid.get_values('q_g_B3'), label='$q_{B3}$')

axes[2].plot(grid.T,grid.get_values('omega_B1'), label='$\omega_{B1}$')
axes[2].plot(grid.T,grid.get_values('omega_B3'), label='$\omega_{B3}$')
axes[2].plot(grid.T,grid.get_values('omega_coi'), label='$\omega_{coi}$')

axes[3].plot([0,t_end],2*[1.05],'r')
#axes[3].plot([0,t_end],2*[0.95],'b')
axes[3].plot(grid.T,grid.get_values('V_B1'), label='$V_{{B1}}$')
axes[3].plot(grid.T,grid.get_values('V_B2'), label='$V_{{B2}}$')
axes[3].plot(grid.T,grid.get_values('V_B3'), label='$V_{{B3}}$')


P_loss = grid.get_values('p_g_B1')*100e3 + grid.get_values('p_g_B3')*100e3 + grid.get_values('P_B2') 
axes[4].plot(grid.T,P_loss, label='$P_{{loss}}$ (W)')

for ax in axes:
    ax.grid()
    ax.legend()
    ax.set_xlim(0,t_end)
ax.set_xlabel('Time (s)')

axes[1].legend(loc='best')

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [164]:
opt.x_0

array([1.04596204, 1.04999538])

## Reporter

In [10]:
U_b = 400.0
S_b = 100e3
I_b = S_b/(np.sqrt(3)*U_b)


def report(grid):
    P_B1,P_B2,P_B3 = grid.get_value('p_g_B1')*100,grid.get_value('P_B1')/1e3,grid.get_value('P_B2')/1e3
    Q_B1,Q_B2,Q_B3 = grid.get_value('q_g_B0_1')*100,grid.get_value('Q_B1')/1e3,grid.get_value('Q_B2')/1e3

    U_B0,U_B1,U_B2,U_B3 = grid.get_value('V_B0')*400,grid.get_value('V_B1')*400,grid.get_value('V_B2')*400,grid.get_value('V_B3')*400
    I_03_m = np.abs(grid.get_value('i_d_ref_B0') + 1j*grid.get_value('i_q_ref_B0'))*I_b
    S_B1_m = np.abs(grid.get_value('P_B1') + 1j*grid.get_value('Q_B1'))/100e3
    I_B1_m = S_B1_m/grid.get_value('V_B1')*I_b
    S_B2_m = np.abs(grid.get_value('P_B2') + 1j*grid.get_value('Q_B2'))/100e3
    I_B2_m = S_B2_m/grid.get_value('V_B2')*I_b
    P_loss = P_B0 + P_B1 + P_B2 - 400

    print(f'    P_1      P_2      P_3       Q_1     Q_2      Q_3    P_loss      I_1      I_2     U_1      U_2     U_3') #      & $P_0\,(kW)$ & $P_1\,(kW)$ & $P_2\,(kW)$ & $P_{loss}\,(kW)$ & $i_{0,3}\,(A)$ & $v_{max}\,(V)$ & $v_{min}\,(V)$ \\ \hline
    print(f'{P_B0:7.2f}  {P_B1:7.2f}  {P_B2:7.2f}  {Q_B0:7.2f}  {Q_B1:7.2f}  {Q_B2:7.2f}   {P_loss:7.2f}  {I_B1_m:7.1f}  {I_B2_m:7.1f}  {U_B1:7.1f}  {U_B2:7.1f} {U_B3:7.1f}') 
    


## Optimization problem

In [70]:
grid_pi_opt = oc_3bus_vsg_pi_class()
gt.change_line(grid_pi,'B1','B2',X_km=0.167,R_km=0.287,km=0.1)
gt.change_line(grid_pi,'B2','B3',X_km=0.167,R_km=0.287,km=0.3)
grid_pi_opt.initialize([params_vsg_pi],'xy_0.json',compile=True)
grid_pi_opt.initialization_tol = 1e-10

P_B2 = -100e3
grid_pi_opt.set_value('P_B2',P_B2)


def obj_eval(u):
    grid_pi_opt.load_0('xy_0.json')
    
    v_ref_B1 = u[0]
    v_ref_B3 = u[1]
    grid_pi_opt.set_value('v_ref_B1',v_ref_B1)
    grid_pi_opt.set_value('v_ref_B3',v_ref_B3) 
    
    params_vsg_pi['v_ref_B1'] = v_ref_B1
    params_vsg_pi['v_ref_B3'] = v_ref_B3
    params_vsg_pi['P_B2'] = P_B2
    
    gt.change_line(grid_pi,'B1','B2',X_km=0.167,R_km=0.287,km=0.1)
    gt.change_line(grid_pi,'B2','B3',X_km=0.167,R_km=0.287,km=0.3)
    grid_pi_opt.initialize([params_vsg_pi],'xy_0.json',compile=True)
    
    P_B1 = grid_pi_opt.get_value('p_g_B1')*100e3
    P_B3 = grid_pi_opt.get_value('p_g_B3')*100e3
    
    P_loss = P_B1 + P_B3 + grid_pi_opt.get_value('P_B2')
    return P_loss

In [73]:
x0 = np.array([ 1,1 ])
#SLSQP
bounds = [(0.95,1.05),(0.95,1.05)]
res = sopt.minimize(obj_eval, x0, method='Powell',bounds=bounds, 
               options={})
res
#grid_pi_opt.save_params('opt.json')

   direc: array([[ 1.06709386e-02,  2.11613751e-02],
       [-1.53616589e-06, -1.46384007e-06]])
     fun: 1168.703133449977
 message: 'Optimization terminated successfully.'
    nfev: 142
     nit: 3
  status: 0
 success: True
       x: array([1.04636675, 1.05      ])

In [72]:
res.x

array([1.04636675, 1.05      ])

In [39]:
grid_pi_opt.initialize([params_vsg_pi],'xy_0.json',compile=True)
P_loss = grid_pi_opt.get_value('p_g_B1')*100e3 + grid_pi_opt.get_value('p_g_B3')*100e3 + grid_pi_opt.get_value('P_B2') 
P_loss

286.9984772622847

In [68]:
grid_pi_opt.run([{'t_end': 1.0,'P_B2':-50e3, 'v_ref_B1':res.x[0],'v_ref_B3':res.x[1]}])
grid_pi_opt.run([{'t_end': 5.0,'P_B2':-100e3}])
grid_pi_opt.run([{'t_end': 8.0,'v_ref_B1':1.04636674,'v_ref_B3':1.05}])
grid_pi_opt.run([{'t_end':30.0,'omega_ref_B1':1.011,'omega_ref_B3':1.011}])
#grid.set_value('P_B2', -1e3)
#grid.run([{'t_end':3.0}])
grid_pi_opt.post();

In [59]:
grid_pi.report_u()

P_B1  =  0.00
Q_B1  =  0.00
P_B2  = -50000.00
Q_B2  =  0.00
P_B3  =  0.00
Q_B3  =  0.00
v_ref_B1 =  1.05
p_m_B1 =  0.25
p_c_B1 =  0.00
omega_ref_B1 =  1.01
v_ref_B3 =  1.05
p_m_B3 =  0.25
p_c_B3 =  0.00
omega_ref_B3 =  1.01


In [48]:
#grid_pi.initialize([params_vsg_pi],'xy_0.json',compile=True)

grid_pi.run([{'t_end': 1.0,'P_B2':-50e3, 'v_ref_B1':res.x[0],'v_ref_B3':res.x[1]}])
grid_pi.run([{'t_end': 5.0,'P_B2':-100e3}])
grid_pi.run([{'t_end': 8.0,'v_ref_B1':1.04636674,'v_ref_B3':1.05}])
grid_pi.run([{'t_end':30.0,'omega_ref_B1':1.011,'omega_ref_B3':1.011}])
#grid.set_value('P_B2', -1e3)
#grid.run([{'t_end':3.0}])
grid_pi.post();

In [69]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(4, 5))
axes[0].plot(grid_pi.T,grid_pi.get_values('omega_B1'), label='$\omega_{B1}$')
axes[0].plot(grid_pi.T,grid_pi.get_values('omega_B3'), label='$\omega_{B3}$')
axes[0].plot(grid_pi.T,grid_pi.get_values('omega_coi'), label='$\omega_{coi}$')
axes[1].plot(grid_pi.T,grid_pi.get_values('V_B2'), label='$V_{{B2}}$')

P_loss = grid_pi_opt.get_values('p_g_B1')*100e3 + grid_pi_opt.get_values('p_g_B3')*100e3 + grid_pi_opt.get_values('P_B2') 
axes[2].plot(grid_pi_opt.T,P_loss, label='$P_{{loss}}$ (W)')

for ax in axes:
    ax.grid()
    ax.legend()
ax.set_xlabel('Time (s)')

axes[1].legend(loc='best')

<ipython-input-69-c23dcbe0a565>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(4, 5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
grid = arn_4bus_class()
grid.initialization_tol = 1e-10
gt.change_line(grid,'B0','B3',X_km=0.167,R_km=0.287,km=0.2)
gt.change_line(grid,'B2','B3',X_km=0.167,R_km=0.287,km=0.2)
gt.change_line(grid,'B1','B3',X_km=0.167,R_km=0.287,km=0.3)

grid.set_value('P_B3',-400e3)

u =   np.array([ 50. ,        253.50910792  , 3.4676086 ,  17.1092196 ])*1e3

grid.set_value('P_B1',u[0])
grid.set_value('P_B2',u[1]) 
grid.set_value('Q_B1',u[2])
grid.set_value('Q_B2',u[3]) 
grid.set_value('K_delta_B0',1)
grid.set_value('R_v_B0',1e-8)
grid.set_value('X_v_B0',1e-8)
grid.load_0('xy_0.json')
grid.ss()
print(grid.get_value('V_B0'))


def obj_eval(u):
    grid_pi.load_0('xy_0.json')
    
    v_ref_B1 = u[0]
    v_ref_B3 = u[1]
    grid.set_value('v_ref_B1',v_ref_B1)
    grid.set_value('v_ref_B3',v_ref_B3) 
    
    grid.ss()
    
    P_B1 = grid_pi.get_values('p_g_B1')*100e3
    P_B1 = grid_pi.get_values('p_g_B1')*100e3
    
    P_loss = P_B0 + P_B1 + P_B2 - 400e3
    return P_loss

def contraint_I_B0_B3(u):
    grid.set_value('P_B1',u[0])
    grid.set_value('P_B2',u[1]) 
    grid.set_value('Q_B1',u[2])
    grid.set_value('Q_B2',u[3]) 
    grid.load_0('xy_0.json')
    grid.ss()
    
    I_B0_B3 = np.abs(grid.get_value('i_d_ref_B0') + 1j*grid.get_value('i_q_ref_B0'))*I_b
    
    return I_B0_B3

def contraint_I_B1(u):
    grid.set_value('P_B1',u[0])
    grid.set_value('P_B2',u[1]) 
    grid.set_value('Q_B1',u[2])
    grid.set_value('Q_B2',u[3]) 
    grid.load_0('xy_0.json')
    grid.ss()
    
    S_B1_m = np.abs(grid.get_value('P_B1') + 1j*grid.get_value('Q_B1'))/100e3
    I_B1_m = S_B1_m/(np.sqrt(3)*grid.get_value('V_B1')*400)
    
    return I_B1_m

def contraint_I_B2(u):
    grid.set_value('P_B1',u[0])
    grid.set_value('P_B2',u[1]) 
    grid.set_value('Q_B1',u[2])
    grid.set_value('Q_B2',u[3]) 
    grid.load_0('xy_0.json')
    grid.ss()
    
    S_B2_m = np.abs(grid.get_value('P_B2') + 1j*grid.get_value('Q_B2'))
    I_B2_m = S_B2_m/(np.sqrt(3)*grid.get_value('V_B2')*400)
    
    return I_B2_m


def contraint_V_B3(u):
    grid.set_value('P_B1',u[0])
    grid.set_value('P_B2',u[1]) 
    grid.set_value('Q_B1',u[2])
    grid.set_value('Q_B2',u[3]) 
    grid.load_0('xy_0.json')
    grid.ss()
    
    U_B3 = grid.get_value('V_B3')*400
    
    return U_B3

def contraint_V_B2(u):
    grid.set_value('P_B1',u[0])
    grid.set_value('P_B2',u[1]) 
    grid.set_value('Q_B1',u[2])
    grid.set_value('Q_B2',u[3]) 
    grid.load_0('xy_0.json')
    grid.ss()
    
    U_B2 = grid.get_value('V_B2')*400
    
    return U_B2


def contraint_V_B1(u):
    grid.set_value('P_B1',u[0])
    grid.set_value('P_B2',u[1]) 
    grid.set_value('Q_B1',u[2])
    grid.set_value('Q_B2',u[3]) 
    grid.load_0('xy_0.json')
    grid.ss()
    
    U_B1 = grid.get_value('V_B1')*400
    
    return U_B1

c1_nlcs = (
    NonlinearConstraint(contraint_I_B1, -80, 80),
    NonlinearConstraint(contraint_I_B2, -476, 476),
   # NonlinearConstraint(contraint_V_B1, 400*0.8, 400*1.2),
   # NonlinearConstraint(contraint_V_B2, 400*0.8, 400*1.2),
   # NonlinearConstraint(contraint_V_B3, 400*0.8, 400*1.2),
   # NonlinearConstraint(contraint_I_B0_B3, -400, 400),
)

c2_nlcs = (
    NonlinearConstraint(contraint_I_B1, -80, 80),
    NonlinearConstraint(contraint_I_B2, -476, 476),
    NonlinearConstraint(contraint_V_B1, 400*0.95, 400*1.05),
    NonlinearConstraint(contraint_V_B2, 400*0.95, 400*1.05),
    NonlinearConstraint(contraint_V_B3, 400*0.95, 400*1.05),
    #NonlinearConstraint(contraint_I_B0_B3, -400, 400),
)

c3_nlcs = (
    NonlinearConstraint(contraint_I_B1, -80, 80),
    NonlinearConstraint(contraint_I_B2, -476, 476),
    NonlinearConstraint(contraint_V_B1, 400*0.95, 400*1.05),
    NonlinearConstraint(contraint_V_B2, 400*0.95, 400*1.05),
    #NonlinearConstraint(contraint_V_B3, 400*0.95, 400*1.05),
    NonlinearConstraint(contraint_I_B0_B3, -180, 180),
)

bounds=[(0e3,50e3),(0,400e3),(-50e3,50e3),(-300e3,300e3)]



NameError: name 'arn_4bus_class' is not defined

### Minimize

In [28]:
x0 = np.array([ 50. ,        253.50910792  , 3.4676086 ,  17.1092196 ])*1e3
res = sopt.minimize(obj_eval, x0, method='Powell', 
               options={})
report(grid)

    P_0      P_1      P_2       Q_0     Q_1      Q_2    P_loss      I_1      I_2     I_03         U_1      U_2     U_3
 140.95   113.22   169.86    -4.80     7.29    11.49     24.03    161.7    242.6    203.6       405.1    405.1   380.3


### Case 1

In [30]:
c1_solution = sopt.differential_evolution(obj_eval,bounds=bounds ,constraints=(c1_nlcs), tol=1e-10)
report(grid)

    P_0      P_1      P_2       Q_0     Q_1      Q_2    P_loss      I_1      I_2     I_03         U_1      U_2     U_3
 171.53    50.00   205.23    -3.52     4.51    14.58     26.76     74.8    292.7    247.7       387.6    405.8   375.9


### Case 2

In [48]:
c2_solution = sopt.differential_evolution(obj_eval, bounds=bounds,constraints=(c2_nlcs), tol=1e-8)
report(grid)

    P_0      P_1      P_2       Q_0     Q_1      Q_2    P_loss      I_1      I_2     I_03         U_1      U_2     U_3
 151.13    50.00   225.98   -16.66     8.94    23.49     27.11     74.8    317.6    219.5       392.1    413.0   380.0


### Case 3

In [49]:
c3_solution = sopt.differential_evolution(obj_eval, bounds=bounds,constraints=(c3_nlcs), tol=1e-8)
report(grid)

    P_0      P_1      P_2       Q_0     Q_1      Q_2    P_loss      I_1      I_2     I_03         U_1      U_2     U_3
 124.61    50.00   253.53    -4.85     3.83    17.39     28.15     73.5    350.6    180.0       394.1    418.4   382.7
